## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_data/WeatherPy_database.csv")
#school_data_to_load = os.path.join("Weather_Database", "WeatherPy_database.csv")
#city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,57,0,9.22,clear sky
1,1,Hithadhoo,MV,-0.6000,73.0833,84.61,77,39,20.02,scattered clouds
2,2,Menongue,AO,-14.6585,17.6910,63.14,54,2,4.94,clear sky
3,3,Dingle,PH,10.9995,122.6711,77.54,89,100,7.49,overcast clouds
4,4,Cape Town,ZA,-33.9258,18.4232,57.61,83,0,4.61,clear sky


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? (°F) "))
max_temp = float(input("What is the maximum temperature you would like for your trip? (°F) "))

What is the minimum temperature you would like for your trip? (°F) 70
What is the maximum temperature you would like for your trip? (°F) 80


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,57,0,9.22,clear sky
3,3,Dingle,PH,10.9995,122.6711,77.54,89,100,7.49,overcast clouds
5,5,Rafai,CF,4.9500,23.9167,72.66,90,100,6.76,light rain
7,7,Victoria,HK,22.2855,114.1577,77.38,90,100,13.00,light rain
9,9,Ovalle,CL,-30.5983,-71.2003,78.66,29,6,4.97,clear sky
...,...,...,...,...,...,...,...,...,...,...
681,681,Baturaja,ID,-4.1333,104.1667,75.49,95,99,2.64,overcast clouds
685,685,Pemangkat,ID,1.1667,108.9667,77.99,85,100,3.85,overcast clouds
695,695,Conakry,GN,9.5380,-13.6773,78.87,83,75,4.61,haze
697,697,Okitipupa,NG,6.5047,4.7839,78.76,91,46,4.14,scattered clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                166
City                   166
Country                165
Lat                    166
Lng                    166
Max Temp               166
Humidity               166
Cloudiness             166
Wind Speed             166
Current Description    166
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,
3,Dingle,PH,77.54,overcast clouds,10.9995,122.6711,
5,Rafai,CF,72.66,light rain,4.9500,23.9167,
7,Victoria,HK,77.38,light rain,22.2855,114.1577,
9,Ovalle,CL,78.66,clear sky,-30.5983,-71.2003,
17,Vila Do Maio,CV,77.79,scattered clouds,15.1333,-23.2167,
18,Utete,TZ,72.50,scattered clouds,-7.9860,38.7580,
19,Saint Anthony,US,74.05,overcast clouds,45.0205,-93.2180,
29,Rikitea,PF,76.17,scattered clouds,-23.1203,-134.9692,
33,Ca Mau,VN,79.16,overcast clouds,9.1769,105.1500,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters. Use the search term: "lodging" 
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(str(index) + "Hotel not found... skipping.")
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
     

5Hotel not found... skipping.
76Hotel not found... skipping.
84Hotel not found... skipping.
102Hotel not found... skipping.
185Hotel not found... skipping.
223Hotel not found... skipping.
308Hotel not found... skipping.
361Hotel not found... skipping.
370Hotel not found... skipping.
402Hotel not found... skipping.
445Hotel not found... skipping.
510Hotel not found... skipping.
529Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [25]:
hotel_df.count()

City                   166
Country                165
Max Temp               166
Current Description    166
Lat                    166
Lng                    166
Hotel Name             166
dtype: int64

In [26]:
hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
3,Dingle,PH,77.54,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
5,Rafai,CF,72.66,light rain,4.9500,23.9167,
7,Victoria,HK,77.38,light rain,22.2855,114.1577,Mini Hotel Central
9,Ovalle,CL,78.66,clear sky,-30.5983,-71.2003,Hotel Altos de Tuqui
17,Vila Do Maio,CV,77.79,scattered clouds,15.1333,-23.2167,Hotel Marilu
18,Utete,TZ,72.50,scattered clouds,-7.9860,38.7580,River View Lodge
19,Saint Anthony,US,74.05,overcast clouds,45.0205,-93.2180,Holiday Inn Express Roseville-St. Paul
29,Rikitea,PF,76.17,scattered clouds,-23.1203,-134.9692,People ThankYou
33,Ca Mau,VN,79.16,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau


In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
3,Dingle,PH,77.54,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
5,Rafai,CF,72.66,light rain,4.9500,23.9167,NaN
7,Victoria,HK,77.38,light rain,22.2855,114.1577,Mini Hotel Central
9,Ovalle,CL,78.66,clear sky,-30.5983,-71.2003,Hotel Altos de Tuqui
...,...,...,...,...,...,...,...
681,Baturaja,ID,75.49,overcast clouds,-4.1333,104.1667,"HOTEL Sekar Sunja, Baturaja Timur, Ogan Komeri..."
685,Pemangkat,ID,77.99,overcast clouds,1.1667,108.9667,Ramah Kost KITA
695,Conakry,GN,78.87,haze,9.5380,-13.6773,Riviera Royal Hotel
697,Okitipupa,NG,78.76,scattered clouds,6.5047,4.7839,Coastal Hotel & Suites


In [28]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
3,Dingle,PH,77.54,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
7,Victoria,HK,77.38,light rain,22.2855,114.1577,Mini Hotel Central
9,Ovalle,CL,78.66,clear sky,-30.5983,-71.2003,Hotel Altos de Tuqui
17,Vila Do Maio,CV,77.79,scattered clouds,15.1333,-23.2167,Hotel Marilu
...,...,...,...,...,...,...,...
681,Baturaja,ID,75.49,overcast clouds,-4.1333,104.1667,"HOTEL Sekar Sunja, Baturaja Timur, Ogan Komeri..."
685,Pemangkat,ID,77.99,overcast clouds,1.1667,108.9667,Ramah Kost KITA
695,Conakry,GN,78.87,haze,9.5380,-13.6773,Riviera Royal Hotel
697,Okitipupa,NG,78.76,scattered clouds,6.5047,4.7839,Coastal Hotel & Suites


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a heatmap of temperature for the vacation spots and marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))